In [2]:
from Models.SolarSTEPS import SolarSTEPS
from Models.ProbabilisticAdvection import ProbabilisticAdvection
from pysteps.motion.lucaskanade import dense_lucaskanade
from compute_metrics import compute_ensemble_metrics
import numpy as np

In [ ]:
# import input images and output images to use to evaluate the forecast
input_images = ...
output_images = ...

In [ ]:
# compute Lucas-Kanade motion field
motion_field = dense_lucaskanade(input_images)

In [ ]:
# initialize the models
pa = ProbabilisticAdvection(alpha=9.23, # variance of norm perturbation
                            beta=0.14, # k value for vonMises distribution for angle perturbation
                            ens_members=40,
                            return_motion_field=False)

solarsteps = SolarSTEPS(ar_order=1,
                        n_cascade_levels=7,
                        probmatching=True,
                        norm=True,
                        local=False,
                        noise_kwargs={'noise_win_size':90,
                                      'noise_std_win_size':15,
                                      'noise_method':'local-SSFT'},
                        norm_kwargs={'extra_normalization':True},
                        verbose=False
                        )

In [ ]:
n_steps = 16

# compute probabilsitic advection forecast ensemble
pa_yhat = pa.maps_forecast(n_steps, input_images, motion_field)

# preprocess images in case of discontinuities
np.random.seed(0)
input_images += np.random.uniform(-0.00001, 0.00001, input_images.shape)

# compute SolarSTEPS forecast ensemble
ss_yhat = solarsteps.ensemble_forecast(input_images, motion_field,
                                       n_steps, seeds=np.arange(40))

In [ ]:
# compute the metrics
solarsteps_result = compute_ensemble_metrics(ss_yhat, output_images)
pa_results = compute_ensemble_metrics(pa_yhat, output_images)